# Run marketing campaigns on target customer segments 
### <font color='blue'>Integrate with Watson Campaign Automation.</font>

## 1. Setup
To prepare your environment, you need to install some packages.

### 1.1 Install the necessary packages

You need the latest versions of these packages:<br>
- ibm-cos-sdk: is a client for the Cloud Object Storage.<br>

** Install the cloud object storage client: **

In [ ]:
!pip install ibm-cos-sdk

### 1.2 Import packages and libraries

Import the packages and libraries that you'll use:

In [ ]:
import ibm_boto3
import pandas as pd
from botocore.client import Config
import types,requests, json
import datetime

## 2 Configure and access data files from Cloud Object Storage

### 2.1 Insert credentials

In [ ]:
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IBM_API_KEY_ID': '',
    'IAM_SERVICE_ID': '',
    'ENDPOINT': '',
    'IBM_AUTH_ENDPOINT': '',
    'BUCKET': '',
    'FILE': ''
}


### 2.2 Functions to work with Cloud Object Storage

In [ ]:
cos = ibm_boto3.client('s3',
                    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
                    ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
                    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
                    config=Config(signature_version='oauth'),
                    endpoint_url=credentials_1['ENDPOINT'])

def get_file(filename):
    '''Retrieve file from Cloud Object Storage'''
    fileobject = cos.get_object(Bucket=credentials_1['BUCKET'], Key=credentials_1['FILE'])['Body']
    return fileobject

def load_string(fileobject):
    '''Load the file contents into a Python string'''
    text = fileobject.read()
    return text

def put_file(filename, filecontents):
    '''Write file to Cloud Object Storage'''
    resp = cos.put_object(Bucket=credentials_1['BUCKET'], Key=filename, Body=filecontents)
    return resp

def __iter__(self): return 0

## 3. Configure and run campaigns on Watson Campaign Automation

### 3.1 Configure access url, client id, client secret, refresh token and access token for Watson Campaign Automation

In [ ]:
# URL to invoke to generate access token
access_token_url = '<BASE_URL>/oauth/token'
 
# Specify client id, client secret, refresh token
data = {'client_id':'',
        'client_secret':'',
        'refresh_token':'',
        'grant_type':'refresh_token'}
 
# Invoke Watson Campaign Automation to generate access token    
r = requests.post(url = access_token_url, data = data)
access_token = json.loads(r.text)["access_token"]

### 3.2 Functions to add contacts and run e-mail campaigns on Watson Campaign Automation

In [ ]:
# XML API url
xmlapiurl = '<BASE_URL>/XMLAPI'

# requuest headers
headers = {'Content-Type': 'text/xml', 'Authorization': 'Bearer ' + access_token, 'Accept':'text/html,application/xhtml+xml,application/xml'}

# Database ID
databaseid = 88308

# Contact List ID for running campaigns
campaignlistid = 88309

# Template ID
templateid = 461421

# Product to run campaign
product = 'Canned Foods'

def add_contact(emailid, databaseid, campaignlistid):
    addcontactxml ="""<Envelope>
                      <Body>
                          <AddRecipient>
                             <LIST_ID>"""+databaseid +"""</LIST_ID>
                             <CREATED_FROM>1</CREATED_FROM>
                             <CONTACT_LISTS>
                                <CONTACT_LIST_ID>"""+campaignlistid+"""</CONTACT_LIST_ID>
                              </CONTACT_LISTS>
                             <COLUMN>
                               <NAME>EMAIL</NAME>
                               <VALUE>"""+emailid+"""</VALUE>
                             </COLUMN>
                          </AddRecipient>
                       </Body>
                    </Envelope>"""
    print (addcontactxml)
    response = requests.post(xmlapiurl, headers=headers, data = addcontactxml)
    print(response.text)
    
def run_campaign(templateid,campaignlistid, campaignname):
    requestxml = """<Envelope><Body>
              <ScheduleMailing>
              <TEMPLATE_ID>"""+templateid + """</TEMPLATE_ID>
              <LIST_ID>"""+campaignlistid+"""</LIST_ID>
              <MAILING_NAME>"""+campaignname+"""</MAILING_NAME>
              <SEND_HTML/>
              <VISIBILITY>1</VISIBILITY>
              </ScheduleMailing>
           </Body></Envelope>"""
    print(requestxml)
    response = requests.post(xmlapiurl, headers=headers, data = requestxml)
    print(response.text)

## 4. Find target customers for running campaigns

### 4.1 Load customer sales data from Cloud object storage

In [ ]:
body = get_file(credentials_1["FILE"])
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
salesData = pd.read_csv(body)
print(salesData.columns)
salesData.head()

### 4.2 Prepare data

In [ ]:
# Filter columns 
temp_data_1 = salesData[['CUSTNAME','CUST_ID','EMAIL_ADDRESS','ORDER_ID','ORDER_DATE','ORDER_VALUE', product]]

# Filter rows 
temp_data_2 = temp_data_1[temp_data_1[product] == 1]


# Change the order dates to make it more recent
temp_data_2['ORDER_DATE'] = temp_data_2['ORDER_DATE'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S.000").date())

datemax = max(temp_data_2['ORDER_DATE'])
currentdate = datetime.date.today()
delta = currentdate - datemax
temp_data_2['ORDER_DATE'] = temp_data_2['ORDER_DATE'].apply(lambda x: x+delta)
temp_data_2

### 4.3 Recency, Frequency and Monetary Value for Customers 

In [ ]:
rfm_df = temp_data_2.groupby('CUST_ID').agg({'CUSTNAME': lambda x: x.unique(),'EMAIL_ADDRESS': lambda x: x.unique(),'ORDER_DATE': lambda x: (datetime.date.today() - x.max()).days, 'ORDER_ID': lambda x: len(x), 'ORDER_VALUE': lambda x: x.sum()})

rfm_df['ORDER_DATE'] = rfm_df['ORDER_DATE'].astype(int)
rfm_df.rename(columns={'ORDER_DATE': 'Recency', 
                       'ORDER_ID': 'Frequency', 
                       'ORDER_VALUE': 'Monetary_Value'}, inplace=True)

### 4.4 Run the campaign for top 5 customers based on Monetary Value, Frequency and Recency
The criteria to run campaigns can be changed based on need

In [ ]:
rfm_df = rfm_df.sort_values(['Monetary_Value','Frequency', 'Recency'], ascending=[0,0,0])
top5 = rfm_df.head()
top5

In [ ]:
# Configure database id, campaign list id and template id in Sec 3.2
for index, row in top5.iterrows():
    add_contact(row['EMAIL_ADDRESS'], str(databaseid), str(campaignlistid))
 
run_campaign(str(templateid), str(campaignlistid), "Offer on canned foods this month!")